In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os
import google.generativeai as palm
from google.api_core import client_options as client_options_lib
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = os.getenv('SERPER_API_KEY')

In [5]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

Inserting batches in chromadb: 100%|██████████████| 1/1 [00:01<00:00,  1.26s/it]


In [6]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [7]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [8]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [10]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

In [11]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [13]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [14]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [15]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


In [18]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

In [27]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [35]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

In [37]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Summary
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, where he has excelled in leading both remote and in-office engineering teams. His expertise spans across software development, process innovation, and enhancing team collaboration. Proficient in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, alongside deep expertise in various front end frameworks. Significant experience in data science and machine learning has enabled him to spearhead successful deployments of scalable AI solutions and innovative data model development.

## Work Experience

### Director of Software Engineering at DataKernel (remote) — 2022 - Present
- Transformed the engineering division into a key revenue pillar, rapidly expanding the customer base and enhancing product capabilities with AI technologies and scalable vector databases.
- Led the team to achieve strategic project goals and set new benchmarks within the industry.

### Senior Software Engineering Manager at DataKernel (remote) — 2019 - 2022
- Directed engineering strategy and operations, managing diverse teams across multiple time zones and fostering a culture of continuous improvement and high performance.

### Founder & CEO at InnovPet (remote) — 2019 - 2022
- Founded a startup focused on IoT solutions for pet care, overseeing product development and navigating the company through a successful initial funding phase.

### Engineering Manager at EliteDevs (remote) — 2018 - 2019
- Formulated and executed strategic plans that enhanced inter-departmental coordination and trust, leading to better project outcomes.

### Engineering Manager at PrintPack (remote) — 2016 - 2018
- Led the development of a high-performance engineering team, integrating data analytics into business decision-making processes.

### Senior Software Engineer at DriveAI (remote) — 2015 - 2016
- Developed and optimized a central API that improved system performance and user satisfaction.

### CTO at BetCraft — 2013 - 2015
- Led technology strategy post-Series A funding, improving platform performance and expanding market reach.

## Education
- MBA in Information Technology from London Business School
- Advanced Leadership Techniques Certification from University of London
- Data Science Specialization Certification from Coursera (Johns Hopkins University)
- B.Sc. in Computer Science from University of Edinburgh

Noah Williams is an ideal candidate for senior executive roles, particularly in companies seeking leadership with a robust blend of technical and strategic expertise.

In [39]:
display(Markdown("./interview_materials.md"))

Based on Noah Williams' extensive experience and expertise in software engineering, team management, and leadership, here are some tailored interview questions and talking points to help him highlight his qualifications and match them with the job requirements:

1. Can you elaborate on your experience as a Full Stack Developer and how it has prepared you for managing remote and in-office engineering teams effectively?
2. How have you leveraged your knowledge of multiple front-end languages and libraries to enhance user experiences in desktop and mobile applications?
3. Can you discuss a project where you integrated AI technologies or machine learning models to drive innovation and growth within a tech startup or organization?
4. How do you approach project management and organizational skills to ensure the successful execution of strategic initiatives and major tech initiatives?
5. In what ways have you demonstrated your proficiency in programming languages and frameworks, such as Ruby, Python, JavaScript, TypeScript, and Elixir, to develop impactful software solutions?
6. How have you utilized your knowledge of databases, web servers, and UI/UX design to create scalable and user-friendly applications?
7. Can you share an example of a project where you implemented data privacy regulations and best practices to ensure compliance and data security?
8. How do you approach technical discussions with both engineering teams and business stakeholders to bridge the gap between technical requirements and business objectives?
9. Have you had experience working with big data tools like Hadoop, Spark, or Kafka, and if so, how have you utilized them to drive data-driven decision-making processes?
10. How do you stay up-to-date with emerging technologies and trends in software development to drive continuous innovation within your teams and projects?

Talking Points:
- Highlight your experience in leading major tech initiatives and startups, showcasing your ability to drive innovation and growth.
- Emphasize your expertise in software engineering, team management, and strategic leadership, aligning with the job requirements for a Full Stack Developer role.
- Discuss your project experiences, contributions to open-source projects, and interest in AI-driven solutions to demonstrate your passion for technology and innovation.
- Share examples of your collaboration and communication style, focusing on results-driven, visionary, and collaborative approaches to problem-solving.
- Mention your educational background, certifications, and industry affiliations to showcase your commitment to continuous learning and professional development.

These questions and talking points will help Noah Williams effectively communicate his qualifications, experiences, and skills during the interview process, highlighting his fit for the job requirements and demonstrating his value as a seasoned leader in the tech industry.